In [0]:
from __future__ import print_function
from __future__ import division
%matplotlib inline

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

In [0]:
train_features = pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_features_train.csv',
                             index_col=[0,1,2])
train_features.interpolate(inplace=True)
train_features.drop('week_start_date', axis=1, inplace=True)
scaler = StandardScaler()
train_features[train_features.columns] = scaler.fit_transform(train_features)


train_labels = pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_labels_train.csv',
                           index_col=[0,1,2])

In [0]:
# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_features.rolling(window=30,min_periods=1)
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_features.rolling(window=30,min_periods=1)
iq_train_labels = train_labels.loc['iq']


In [24]:
from sklearn.feature_selection import RFE

# create a base classifier used to evaluate a subset of attributes
model = LinearRegression()

# create the RFE model and select n attributes
rfe = RFE(model,4)
rfe = rfe.fit(sj_train_features, sj_train_labels)
X_Select = sj_train_features.loc[:, rfe.support_]
print(rfe.support_)

# Split based on the newly selected features and train and test again
X_train, X_test, Y_train, Y_test = train_test_split(X_Select, sj_train_labels, shuffle = False)
model.fit(X_train, Y_train)
print(model.score(X_train, Y_train))
Y_pred = model.predict(X_test)

mean_absolute_error(Y_test, Y_pred)



[False False False False False  True False  True False False False  True
 False False  True False False False False False]
0.03800500000378837


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


24.69597466324993

In [25]:
clf = linear_model.Lasso(alpha=0.0001)
history =clf.fit(X_train, Y_train)
# print(Y_train)
print(clf.score(X_train, Y_train))
Y_pred = clf.predict(X_test)
print(clf.score(X_test, Y_test))
mean_absolute_error(Y_test, Y_pred)
print(Y_test)
print(Y_pred.astype(int))

0.035594304886770844
-0.08669888145714744
                 total_cases
year weekofyear             
2003 44                   39
     45                   39
     46                   34
     47                   30
     48                   37
     49                   29
     50                   26
     51                   15
     52                   22
2004 1                    15
     2                    20
     3                    14
     4                    10
     5                    21
     6                    14
     7                    14
     8                     9
     9                    11
     10                    5
     11                    6
     12                    7
     13                   11
     14                    4
     15                    3
     16                    2
     17                    6
     18                   10
     19                    7
     20                    5
     21                    3
...                      ...
2

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1071349.908321353, tolerance: 223.6617948717949
  positive)


In [26]:
test_features = pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_features_test.csv',
                             index_col=[0,1,2])
test_features.interpolate(inplace=True)
test_features.drop('week_start_date', axis=1, inplace=True)
scaler = StandardScaler()
test_features[test_features.columns] = scaler.fit_transform(test_features)
test_features.rolling(window=90,min_periods=1)
sj_test_features = test_features.loc['sj']
X_Select_test = sj_test_features.loc[:, rfe.support_]
sj_pred = clf.predict(X_Select_test).astype(int)
print(sj_pred)


[29 34 21 21 28 35 34 30 40 46 45 41 43 40 45 43 45 43 39 42 38 41 38 39
 35 39 40 42 34 40 41 33 39 38 30 29 27 39 34 37 29 33 30 30 28 35 28 17
 26 25 25 26 36 30 43 42 42 38 38 42 38 32 41 41 42 43 45 46 45 42 42 44
 36 39 40 45 42 44 39 38 42 35 39 38 43 43 36 34 32 31 43 34 37 22 21 34
 28 32 31 30 33 32 32 37 42 36 41 45 36 48 41 44 48 45 45 46 45 50 42 46
 40 45 39 43 48 38 43 42 43 35 38 40 47 33 40 41 26 26 31 37 38 38 39 30
 28 32 31 32 30 27 27 26 33 34 30 35 28 44 37 40 45 49 49 52 51 50 50 50
 48 49 47 51 46 52 48 45 48 46 48 47 46 45 41 44 40 43 45 44 41 45 36 38
 36 37 34 33 29 32 29 29 30 31 31 28 36 36 35 37 32 41 42 44 40 35 37 34
 36 38 42 47 44 47 45 49 46 46 46 43 36 41 46 47 43 48 46 41 45 42 45 40
 40 39 35 36 38 38 28 35 30 29 28 28 22 32 24 28 33 31 31 36]


In [27]:
from sklearn.feature_selection import RFE

# create a base classifier used to evaluate a subset of attributes
model = LinearRegression()

# create the RFE model and select n attributes
rfe = RFE(model, 4)
rfe = rfe.fit(iq_train_features, iq_train_labels)
X_Select = iq_train_features.loc[:, rfe.support_]
# Split based on the newly selected features and train and test again
X_train, X_test, Y_train, Y_test = train_test_split(X_Select, iq_train_labels, shuffle = False)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

mean_absolute_error(Y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


7.8372963084632

In [28]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
mean_absolute_error(Y_test, Y_pred)

8.035810632959045

In [29]:
test_features = pd.read_csv('https://s3.amazonaws.com/drivendata/data/44/public/dengue_features_test.csv',
                             index_col=[0,1,2])
test_features.interpolate(inplace=True)
test_features.drop('week_start_date', axis=1, inplace=True)
scaler = StandardScaler()
test_features[test_features.columns] = scaler.fit_transform(test_features)
test_features.rolling(window=30,min_periods=1)
iq_test_features = test_features.loc['iq']
X_Select_test = iq_test_features.loc[:, rfe.support_]
iq_pred = clf.predict(X_Select_test).astype(int)
print(iq_pred)

[7 5 7 4 4 6 2 5 3 6 6 5 6 5 4 6 7 8 9 7 7 8 8 8 7 5 7 5 7 7 5 6 7 8 8 8 7
 7 8 8 8 7 7 8 8 4 6 7 6 5 6 4 5 5 3 4 5 4 5 2 4 3 4 5 6 7 6 7 8 7 7 7 8 8
 7 8 7 7 8 7 7 7 6 7 8 7 6 6 7 6 6 7 7 8 8 7 7 5 7 5 5 6 5 6 5 5 4 5 2 4 3
 3 5 4 4 5 4 5 5 7 7 8 7 7 8 9 9 8 8 9 8 6 6 8 7 8 7 8 7 7 8 9 8 7 7 8 7 3
 8 7 6 6 7 5 4 5]


In [30]:
submission = pd.read_csv("https://s3.amazonaws.com/drivendata/data/44/public/submission_format.csv",
                         index_col=[0, 1, 2])
submission.total_cases = np.concatenate([sj_pred, iq_pred])
print(submission)
submission.to_csv("submission.csv")
from google.colab import files

files.download('submission.csv')

                      total_cases
city year weekofyear             
sj   2008 18                   29
          19                   34
          20                   21
          21                   21
          22                   28
          23                   35
          24                   34
          25                   30
          26                   40
          27                   46
          28                   45
          29                   41
          30                   43
          31                   40
          32                   45
          33                   43
          34                   45
          35                   43
          36                   39
          37                   42
          38                   38
          39                   41
          40                   38
          41                   39
          42                   35
          43                   39
          44                   40
          45  